In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt
import statistics 
from scipy.stats import *
from scipy.spatial import distance

######### Instance Transfer repositories ####################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Repositories uploaded!!")

Repositories uploaded!!


In [2]:
################################### Parkinsons ###########################################################################################################
## Target Data: motor_UPDRS, total_UPDRS: choose one --> Let's go with total_UPDRS and remove motor UPDRS
## Correlation col: Shimmer
## Cuts at: 0.021 and 0.035
##########################################################################################################################################################

Parkinsons_df = pd.read_csv("Scientific_data/Parkinsons/parkinsons_updrs.data")
motor_col = ['motor_UPDRS']
Parkinsons_df = Parkinsons_df.drop(motor_col, axis = 1)

print("Parkinsons Data")
print("-------------------------------------------")
print(Parkinsons_df.shape)

Parkinsons_df

# # print("The correlation matrix is: ")
# # Parkinsons_df.corr()['total_UPDRS'].abs().sort_values()

# drop_col_parkinsons = ['Shimmer']

# Parkinsons_tgt_df = Parkinsons_df.loc[(Parkinsons_df['Shimmer'] >= 0.021) & (Parkinsons_df['Shimmer'] < 0.035)]
# Parkinsons_tgt_df = Parkinsons_tgt_df.drop(drop_col_parkinsons, axis = 1)
# Parkinsons_tgt_df = Parkinsons_tgt_df.reset_index(drop = True)
# print("Target Set: ",Parkinsons_tgt_df.shape)

# Parkinsons_source_df = Parkinsons_df.loc[(Parkinsons_df['Shimmer'] < 0.021)]
# Parkinsons_source_df = Parkinsons_source_df.drop(drop_col_parkinsons, axis = 1)
# Parkinsons_source_df = Parkinsons_source_df.reset_index(drop = True)
# print("Source Set: ",Parkinsons_source_df.shape)

# Parkinsons_test_df = Parkinsons_df.loc[(Parkinsons_df['Shimmer'] >= 0.035)]
# Parkinsons_test_df = Parkinsons_test_df.drop(drop_col_parkinsons, axis = 1)
# Parkinsons_test_df = Parkinsons_test_df.reset_index(drop = True)
# print("Test Set: ",Parkinsons_test_df.shape)



Parkinsons Data
-------------------------------------------
(5875, 21)


,subject#,age,sex,test_time,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,34.398,0.00662,0.000034,0.00401,0.00317,0.01204,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,34.894,0.00300,0.000017,0.00132,0.00150,0.00395,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,35.389,0.00481,0.000025,0.00205,0.00208,0.00616,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,35.810,0.00528,0.000027,0.00191,0.00264,0.00573,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,36.375,0.00335,0.000020,0.00093,0.00130,0.00278,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,42,61,0,142.7900,33.485,0.00406,0.000031,0.00167,0.00168,0.00500,...,0.160,0.00973,0.01133,0.01549,0.02920,0.025137,22.369,0.64215,0.55314,0.21367
5871,42,61,0,149.8400,32.988,0.00297,0.000025,0.00119,0.00147,0.00358,...,0.215,0.01052,0.01277,0.01904,0.03157,0.011927,22.886,0.52598,0.56518,0.12621
5872,42,61,0,156.8200,32.495,0.00349,0.000025,0.00152,0.00187,0.00456,...,0.244,0.01371,0.01456,0.01877,0.04112,0.017701,25.065,0.47792,0.57888,0.14157
5873,42,61,0,163.7300,32.007,0.00281,0.000020,0.00128,0.00151,0.00383,...,0.131,0.00693,0.00870,0.01307,0.02078,0.007984,24.422,0.56865,0.56327,0.14204


In [28]:
################################### Communities and Crime ###########################################################################################################
## Target Data: 
## Correlation col: Shimmer
## Cuts at: 0.021 and 0.035
##########################################################################################################################################################

communities_df = pd.read_csv("Scientific_data/Communities/communities.data")

In [3]:
################################### Superconductivity ###########################################################################################################
## Target Data: critical_temp
## Correlation col: gmean_ElectronAffinity
## Cuts at: 40.0 and 60.0
##########################################################################################################################################################

superconduct_df = pd.read_csv("Scientific_data/superconduct/train.csv")
superconduct_unq_df = pd.read_csv("Scientific_data/superconduct/unique_m.csv")

print("Superconductivity Data")
print("-------------------------------------------")
print(superconduct_df.shape)

superconduct_df

# # print("The correlation matrix is: ")
# # superconduct_df_corr = superconduct_df.corr()['critical_temp'].abs().sort_values()
# # print(superconduct_df_corr.to_string()) ### Helps to print the entire series

# drop_col_superconduct = ['gmean_ElectronAffinity']
# # superconduct_df['gmean_ElectronAffinity'].sort_values()

# superconduct_tgt_df = superconduct_df.loc[(superconduct_df['gmean_ElectronAffinity'] < 40.0)]
# superconduct_tgt_df = superconduct_tgt_df.drop(drop_col_superconduct, axis = 1)
# superconduct_tgt_df = superconduct_tgt_df.reset_index(drop = True)
# print("Target Set: ",superconduct_tgt_df.shape)

# superconduct_source_df = superconduct_df.loc[(superconduct_df['gmean_ElectronAffinity'] >= 40.0) & (superconduct_df['gmean_ElectronAffinity'] < 60.0)]
# superconduct_source_df = superconduct_source_df.drop(drop_col_superconduct, axis = 1)
# superconduct_source_df = superconduct_source_df.reset_index(drop = True)
# print("Source Set: ", superconduct_source_df.shape)

# superconduct_test_df = superconduct_df.loc[(superconduct_df['gmean_ElectronAffinity'] >= 60.0)]
# superconduct_test_df = superconduct_test_df.drop(drop_col_superconduct, axis = 1)
# superconduct_test_df = superconduct_test_df.reset_index(drop = True)
# print("Test Set: ", superconduct_test_df.shape)


Superconductivity Data
-------------------------------------------
(21263, 82)


,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21259,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21260,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21261,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [78]:
################################### CASP ###########################################################################################################
## Target Data: RMSD
## Correlation col: F6
## Cuts at: 105.0 and 160.0
##########################################################################################################################################################

casp_df = pd.read_csv("Scientific_data/CASP/CASP.csv")

print("CASP Data")
print("-------------------------------------------")
print(casp_df.shape)

# print("The correlation matrix is: ")
# casp_df.corr()['RMSD'].abs().sort_values()

drop_col_casp = ['F6']
# casp_df['F6'].sort_values()

casp_tgt_df = casp_df.loc[(casp_df['F6'] < 105.0)]
casp_tgt_df = casp_tgt_df.drop(drop_col_casp, axis = 1)
casp_tgt_df = casp_tgt_df.reset_index(drop = True)
print("Target Set: ",casp_tgt_df.shape)

casp_source_df = casp_df.loc[(casp_df['F6'] >= 105.0) & (casp_df['F6'] < 160.0)] 
casp_source_df = casp_source_df.drop(drop_col_casp, axis = 1)
casp_source_df = casp_source_df.reset_index(drop = True)
print("Source Set: ", casp_source_df.shape)

casp_test_df = casp_df.loc[(casp_df['F6'] >= 160.0)]
casp_test_df = casp_test_df.drop(drop_col_casp, axis = 1)
casp_test_df = casp_test_df.reset_index(drop = True)
print("Test Set: ", casp_test_df.shape)



CASP Data
-------------------------------------------
(45730, 10)
Target Set:  (15546, 9)
Source Set:  (15195, 9)
Test Set:  (14989, 9)


In [8]:
################################### Superconduct ###########################################################################################################
## Target Data: RMSD
## Correlation col: F6
## Cuts at: 105.0 and 160.0
##########################################################################################################################################################

superconduct_df = pd.read_csv("Scientific_data/superconduct/train.csv")
superconduct_df.index = superconduct_df.index + 1 ### Start index from 1 instead of 0
superconduct_df.columns

########## Corr Superconduct ################
print("The correlation matrix is: ")
print(superconduct_df.corr().sort_values('critical_temp')['critical_temp'])


The correlation matrix is: 
wtd_mean_Valence              -0.632401
wtd_gmean_Valence             -0.615653
mean_Valence                  -0.600085
gmean_Valence                 -0.573068
gmean_Density                 -0.541684
                                 ...   
std_ThermalConductivity        0.653632
range_atomic_radius            0.653759
range_ThermalConductivity      0.687654
wtd_std_ThermalConductivity    0.721271
critical_temp                  1.000000
Name: critical_temp, Length: 82, dtype: float64


In [2]:
######################################################## SARCOS Dataset ########################################################
from scipy.io import loadmat

sarcos_np = train_data = loadmat('UCI_regression/Sarcos/sarcos_inv.mat')['sarcos_inv'].astype(np.float32)
sarcos_df = pd.DataFrame(sarcos_np)

sarcos_train_df = sarcos_df.iloc[: , : -6]
print("Training Set: ", sarcos_train_df.shape)

sarcos_source_df = sarcos_df.iloc[: , : -5]
sarcos_source_df = sarcos_source_df.drop(sarcos_source_df.columns[[22]], axis=1)
print("Testing Set: ", sarcos_source_df.shape)

sarcos_train_df.columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 
                           'C17', 'C18', 'C19', 'C20', 'C21', 'T']

sarcos_source_df.columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 
                           'C17', 'C18', 'C19', 'C20', 'C21', 'T']

print("----------------------------------------------------")

############################################### Standardization ###############################################
sarcos_cols = sarcos_train_df.columns.difference(['T'])
ss = StandardScaler()
sarcos_train_df[sarcos_cols] = ss.fit_transform(sarcos_train_df[sarcos_cols])
sarcos_source_df[sarcos_cols] = ss.transform(sarcos_source_df[sarcos_cols]) ## Use the same scale as the training data


#################### Splitting with small target set and large source and test set #############
sarcos_test_df, sarcos_tgt_df = train_test_split(sarcos_train_df, test_size = 0.05) ## test_size = tgt size

sarcos_train_df = sarcos_train_df.reset_index(drop = True)
sarcos_tgt_df = sarcos_tgt_df.reset_index(drop = True)
sarcos_source_df = sarcos_source_df.reset_index(drop = True)

print("Target Set: ", sarcos_tgt_df.shape)
print("Source Set: ", sarcos_source_df.shape)
print("Test Set: ", sarcos_test_df.shape)

target_sarcos = ['T']

sarcos_df_test_y = sarcos_test_df[target_sarcos]
sarcos_df_test_X = sarcos_test_df.drop(target_sarcos, axis = 1)

sarcos_df_tgt_y = sarcos_tgt_df[target_sarcos]
sarcos_df_tgt_X = sarcos_tgt_df.drop(target_sarcos, axis = 1)

sarcos_df_source_y = sarcos_source_df[target_sarcos]
sarcos_df_source_X = sarcos_source_df.drop(target_sarcos, axis = 1)


############## Merging the datasets #################
sarcos_X_df = pd.concat([sarcos_df_source_X, sarcos_df_tgt_X], ignore_index=True)
sarcos_y_df = pd.concat([sarcos_df_source_y, sarcos_df_tgt_y], ignore_index=True)

sarcos_np_train_X = sarcos_X_df.to_numpy()
sarcos_np_train_y = sarcos_y_df.to_numpy()

sarcos_np_test_X = sarcos_df_test_X.to_numpy()
sarcos_np_test_y = sarcos_df_test_y.to_numpy()

sarcos_np_train_y_list = sarcos_np_train_y.ravel()
sarcos_np_test_y_list = sarcos_np_test_y.ravel()

src_size_sarcos = len(sarcos_df_source_y)
tgt_size_sarcos = len(sarcos_df_tgt_y)

print("-------------------------------------------")
###############################################################################################################################################

Training Set:  (44484, 22)
Testing Set:  (44484, 22)
----------------------------------------------------
Target Set:  (2225, 22)
Source Set:  (44484, 22)
Test Set:  (42259, 22)
-------------------------------------------


<ipython-input-2-16a21a08e964>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarcos_train_df[sarcos_cols] = ss.fit_transform(sarcos_train_df[sarcos_cols])
/Users/shrey/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:2969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/Users/shrey/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:2935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [ ]:
########################### Two-Stage TrAdaBoost.R2 Sarcos #######################################

src_idx_sarcos = np.arange(start = 0, stop = (src_size_sarcos - 1), step = 1)
tgt_idx_sarcos = np.arange(start = src_size_sarcos, stop = ((src_size_sarcos + tgt_size_sarcos)-1), step = 1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}


from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_sarcos = []
rmselist_TwoTrAda_sarcos = []

for x in range(0, 10):

    model_TwoTrAda_sarcos = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_sarcos.fit(sarcos_np_train_X, sarcos_np_train_y_list, src_idx_sarcos, tgt_idx_sarcos)

    y_pred_TwoTrAda_sarcos = model_TwoTrAda_sarcos.predict(sarcos_np_test_X)

    mse_TwoTrAda_sarcos = sqrt(mean_squared_error(sarcos_np_test_y, y_pred_TwoTrAda_sarcos))
    rmselist_TwoTrAda_sarcos.append(mse_TwoTrAda_sarcos)
        
    r2_score_TwoTrAda_sarcos = pearsonr(sarcos_np_test_y_list, y_pred_TwoTrAda_sarcos)
    r2_score_TwoTrAda_sarcos = (r2_score_TwoTrAda_sarcos[0])**2
    r2scorelist_TwoTrAda_sarcos.append(r2_score_TwoTrAda_sarcos)

print("Mean, STDev of RMSE:", rmselist_TwoTrAda_sarcos)
print("Mean, STDev of R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_sarcos)

print("\n")

print("RMSE List of TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_sarcos))
print("R^2 List of TrAdaboost.R2:", statistics.mean(r2scorelist_TwoTrAda_sarcos))


print("-------------------------------------------")

In [3]:
########################### AdaBoostR2 Transfer Learning Sarcos #####################################################
from sklearn.ensemble import AdaBoostRegressor

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_sarcos = []
rmselist_AdaTL_sarcos = []

for x in range(0, 10):

    model_AdaTL_sarcos = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_sarcos.fit(sarcos_np_train_X, sarcos_np_train_y_list)

    y_pred_AdaTL_sarcos = model_AdaTL_sarcos.predict(sarcos_np_test_X)

    mse_AdaTL_sarcos = sqrt(mean_squared_error(sarcos_np_test_y, y_pred_AdaTL_sarcos))
    rmselist_AdaTL_sarcos.append(mse_AdaTL_sarcos)
        
    r2_score_AdaTL_sarcos = pearsonr(sarcos_np_test_y_list, y_pred_AdaTL_sarcos)
    r2_score_AdaTL_sarcos = (r2_score_AdaTL_sarcos[0])**2
    r2scorelist_AdaTL_sarcos.append(r2_score_AdaTL_sarcos)

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_sarcos)
print("R^2 List of Adaboost.R2:", r2scorelist_AdaTL_sarcos)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_sarcos))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_sarcos))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [5.829158217774636, 5.841351157766762, 5.833818322417791, 5.842069537428835, 5.815471428748136, 5.800302760846721, 5.800645076118142, 5.797455064985382, 5.83211923571742, 5.826725419517985]
R^2 List of Adaboost.R2: [0.9215015424142335, 0.921141031096351, 0.9212864110921958, 0.920964980560276, 0.9217230028566479, 0.9223369760958089, 0.9221779629616282, 0.9223675011192882, 0.921165198316636, 0.921517759275278]


Mean, STDev of RMSE: 5.821911622132181
Mean, STDev of R^2: 0.9216182365788343
-------------------------------------------


In [40]:
######################### Gradient Boosting Regression Transfer Learning Sarcos #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_sarcos = []
rmselist_GBRTL_sarcos = []

for x in range(0, 10):

    model_GBRTL_sarcos = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_sarcos.fit(sarcos_np_train_X, sarcos_np_train_y_list)

    y_pred_GBRTL_sarcos = model_GBRTL_sarcos.predict(sarcos_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_sarcos = sqrt(mean_squared_error(sarcos_np_test_y, y_pred_GBRTL_sarcos))
    rmselist_GBRTL_sarcos.append(mse_GBRTL_sarcos)
        
    r2_score_GBRTL_sarcos = pearsonr(sarcos_np_test_y_list, y_pred_GBRTL_sarcos)
    r2_score_GBRTL_sarcos = (r2_score_GBRTL_sarcos[0])**2
    r2scorelist_GBRTL_sarcos.append(r2_score_GBRTL_sarcos)

print("RMSE of GBRTL:", rmselist_GBRTL_sarcos)
print("R^2 of GBRTL:", r2scorelist_GBRTL_sarcos)

print("\n")

    
print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_sarcos))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_sarcos))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


RMSE of GBRTL: [2.436028222372737, 2.436028222372737, 2.436028222372737, 2.436028222372737, 2.436028222372737]
R^2 of GBRTL: [0.9858366818354892, 0.985836681835489, 0.9858366818354892, 0.9858366818354892, 0.9858366818354892]


RMSE of GBRTL: 2.436028222372737
R^2 of GBRTL: 0.9858366818354892
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [31]:
############################################################## Wine Quality Dataset ##############################################################

colnames_wine = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide',
                 'total_sulfur_dioxide', 'density', 'pH', 'sulphates',  'alcohol', 'quality']

wine_red_df = pd.read_csv('UCI_regression/WineQuality/winequality-red.csv', sep = ';') 
wine_red_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
